# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [9]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [10]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [11]:
!pip install langchain langchain-core langchain-openai langchain-community langchain-classic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [12]:
!pip show langchain langchain-core langchain-openai langchain-community

Name: langchain
Version: 1.2.10
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 
---
Name: langchain-core
Version: 1.2.15
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: jsonpatch, langsmith, packaging, pydantic, pyyaml, tenacity, typing-extensions, uuid-utils
Required-by: langchain, langchain-classic, langchain-community, langchain-openai, langchain-text-splitters, langgraph, langgraph-checkpoint, langgraph-prebuilt
---
Name: langchain-openai
Version: 1.1.10
Summary: An integration package connecting OpenAI and LangChain
Home-page: https://docs.langchain.com/oss/python/integrations/providers/openai
Author: 
Author-email: 
License: MIT
Locat

In [13]:
from langchain_openai import ChatOpenAI        # ✅ 1.0.3
from langchain_core.prompts import ChatPromptTemplate  # ✅ From langchain-core 1.0.5
from langchain_classic.chains import LLMChain  # ✅ Legacy (install if missing)

In [18]:
#Using low temperature for more a more factual and consistent answer
llm = ChatOpenAI(temperature=0.3)


In [22]:
prompt = ChatPromptTemplate.from_template(
    "Describe the product: {product}"
)

In [23]:

chain = LLMChain(llm=llm, prompt=prompt)

/tmp/ipython-input-294/546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [25]:
product = "Sportwear"
chain.run(product)

/tmp/ipython-input-294/843948186.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  chain.run(product)


'Sportswear is a type of clothing designed specifically for physical activity and exercise. It typically includes items such as athletic shorts, leggings, tank tops, sports bras, sweat-wicking shirts, and sneakers. Sportswear is made from breathable and moisture-wicking materials to keep the wearer comfortable and dry during workouts. It is often designed with features such as stretchy fabrics, flat seams, and mesh panels for optimal performance and mobility. Sportswear is commonly worn for activities such as running, yoga, weightlifting, and team sports.'

## SimpleSequentialChain

In [26]:
from langchain_classic.chains import SimpleSequentialChain

In [28]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Create a short product title for {product}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)



In [30]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a short product description based on this title: {title}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [31]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [32]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Performance-Driven Sportswear Collection"
Elevate your athletic performance with our Performance-Driven Sportswear Collection. Designed for maximum comfort and functionality, our high-performance activewear is perfect for all your workouts and training sessions. From moisture-wicking fabrics to strategic ventilation, our collection will help you reach your fitness goals in style. Whether you're hitting the gym or hitting the trails, our sportswear will keep you looking and feeling your best.

> Finished chain.


"Elevate your athletic performance with our Performance-Driven Sportswear Collection. Designed for maximum comfort and functionality, our high-performance activewear is perfect for all your workouts and training sessions. From moisture-wicking fabrics to strategic ventilation, our collection will help you reach your fitness goals in style. Whether you're hitting the gym or hitting the trails, our sportswear will keep you looking and feeling your best."

**Repeat the above twice for different products**

## SequentialChain

In [33]:
from langchain_classic.chains import SequentialChain

In [51]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Translate this review into English: {review}"
)

# chain 1
chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt,
    output_key="translated_review"
)

In [53]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize this review: {translated_review}"
)

# chain 2
chain_two = LLMChain(
    llm=llm,
    prompt=second_prompt,
    output_key="title"
)

In [54]:
# prompt template 3: summarize the translated review
third_prompt = ChatPromptTemplate.from_template(
    "Summarize this review in one short sentence: {translated_review}"
)

chain_three = LLMChain(
    llm=llm,
    prompt=third_prompt,
    output_key="summary"
)

In [55]:
# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow-up message based on this summary: {summary}"
)

chain_four = LLMChain(
    llm=llm,
    prompt=fourth_prompt,
    output_key="followup_message"
)


In [61]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["translated_review", "title", "summary", "followup_message"],
    verbose=True
)

In [62]:
review = df.Review[5]
overall_chain(review)

/tmp/ipython-input-294/1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'translated_review': "I find the taste mediocre. The foam doesn't last, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'title': 'The reviewer is not impressed with the taste of the product and finds the foam to be strange and not long-lasting. They suggest that the product may be from an old batch or could be counterfeit because they have had better-tasting versions of the same product purchased in stores.',
 'summary': 'The reviewer is not satisfied with the taste and quality of the product, suspecting it may be a counterfeit or old batch.',
 'followup_message': 'Dear [Reviewer],\n\nThank you for bringing your concerns to our attention regarding the taste and quality of our product. We take these matters very seriously and are committed to providing o

**Repeat the above twice for different products or reviews**

## Router Chain

In [63]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [64]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [65]:
from langchain_classic.chains.router import MultiPromptChain
from langchain_classic.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain_core.prompts import PromptTemplate  # Updated path for prompts (core package)

In [66]:
llm = ChatOpenAI(temperature=0)

In [67]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [68]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [69]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [70]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [71]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

/tmp/ipython-input-294/3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [72]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect absorber of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body at a given wavelength is proportional to the temperature of the body and the wavelength to the power of -5."

In [73]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [74]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures that the organism functions properly as a whole. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and development of all living organisms.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [78]:
review_1 = df.Review.iloc[1]
print("----- REVIEW 1 -----")
result_1 = overall_chain({"review": review_1})
print(result_1)

review_2 = df.Review.iloc[3]
print("----- REVIEW 2 -----")
result_2 = overall_chain({"review": review_2})
print(result_2)

----- REVIEW 1 -----


> Entering new SequentialChain chain...

> Finished chain.
{'review': 'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.', 'translated_review': "I really liked the waterproof bag, although the opening was made of hard plastic. I'm not sure if it could break easily. However, I couldn't turn on my phone once it was inside the pouch.", 'title': 'The reviewer enjoyed using the waterproof bag, but was concerned about the durability of the hard plastic opening. They also experienced difficulty using their phone while it was inside the pouch.', 'summary': 'Reviewer enjoyed the waterproof bag but was concerned about the hard plastic opening potentially breaking and had difficulty using their phone inside the pouch.', 'followup_message': 'Hello,\n\nThank you for your feedback on the waterproof bag. We are glad to hear that you enjoyed using it ove

In [79]:
print(len(df))

7
